In [1]:
# !pip install pandas sqlalchemy openpyxl
# !pip install ipython-sql


In [2]:
%load_ext sql


In [3]:
%sql sqlite:///data_database.db


In [4]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

# Create an SQLite database engine
engine = create_engine("sqlite:///data_database.db")  # Save to file or use ":memory:" for in-memory DB

# List of Excel files and corresponding table names
files = [
    ("BVSP.xlsx", "table_1"),
    ("DJI.xlsx", "table_2"),
    ("FTSE.xlsx", "table_3"),
    ("GDAXI.xlsx", "table_4"),
    ("HSI.xlsx", "table_5"),
    ("NDX.xlsx", "table_6"),
    ("RUT.xlsx", "table_7"),
    ("SPX.xlsx", "table_8"),
]

# Load each Excel file into its own table
for file_path, table_name in files:
    data = pd.read_excel(file_path)
    data.to_sql(table_name, engine, if_exists="replace", index=False)

print("All Excel files have been loaded into SQLite.")


All Excel files have been loaded into SQLite.


In [5]:
# Connect to the SQLite database
conn = sqlite3.connect("data_database.db")

# Define the SQL query with UNION ALL logic
query = """
CREATE TABLE DataTable AS
SELECT Ticker, 
        strftime('%Y-%m-%d', Date) AS Date,
        AdjustedClose,
        Close,
        Volume
FROM(
SELECT 'BVSP' AS Ticker, 
        Date,
        [Adj Close_^BVSP_^BVSP] AS 'AdjustedClose',
        [Close_^BVSP_^BVSP] AS 'Close',
        [Dividends_^BVSP_^BVSP] AS 'Dividends',
        [Stock Splits_^BVSP_^BVSP] AS 'Stock Splits',
        [Volume_^BVSP_^BVSP] AS 'Volume'
FROM table_1
UNION ALL 
SELECT 'DJI' AS Ticker,*
FROM table_2
UNION ALL 
SELECT 'FTSE' AS Ticker,*
FROM table_3
UNION ALL 
SELECT 'GDAXI' AS Ticker,*
FROM table_4
UNION ALL 
SELECT 'HSI' AS Ticker,*
FROM table_5
UNION ALL 
SELECT 'NDX' AS Ticker,*
FROM table_6
UNION ALL 
SELECT 'RUT' AS Ticker,*
FROM table_7
UNION ALL 
SELECT 'SPX' AS Ticker,*
FROM table_8
) AS data;
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

print("Data table created successfully.")


Data table created successfully.


In [6]:
%%sql
SELECT * 
FROM DataTable
LIMIT 10

 * sqlite:///data_database.db
Done.


Ticker,Date,AdjustedClose,Close,Volume
BVSP,2015-01-01,46908.0,46908.0,77719800
BVSP,2015-02-01,51583.0,51583.0,65310100
BVSP,2015-03-01,51150.0,51150.0,79890000
BVSP,2015-04-01,56229.0,56229.0,81114600
BVSP,2015-05-01,52760.0,52760.0,67845700
BVSP,2015-06-01,53081.0,53081.0,61236200
BVSP,2015-07-01,50865.0,50865.0,65571800
BVSP,2015-08-01,46626.0,46626.0,71135200
BVSP,2015-09-01,45059.0,45059.0,79146200
BVSP,2015-10-01,45869.0,45869.0,78617500


In [7]:
# Meta Data Table 

In [8]:
# Connect to the database if not already connected
conn = sqlite3.connect("data_database.db")

# Create a metadata table with columns for ticker, description, currency, and conversion ratio
create_table_query = """
CREATE TABLE ticker_data (
    Ticker TEXT PRIMARY KEY,
    Description TEXT,
    Currency TEXT,
    ConversionToUSD REAL
);
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(create_table_query)
conn.commit()

print("Metadata table created successfully.")


Metadata table created successfully.


In [9]:
# Insert metadata for each ticker with conversion to USD 
# Used conversion rate as of 2024-12-31
insert_data_query = """
INSERT INTO ticker_data (Ticker, Description, Currency, ConversionToUSD)
VALUES 
    ('BVSP', 'Brazilian Stock Index', 'BRL', 0.17),
    ('DJI', 'Dow Jones Industrial Average', 'USD', 1.00),
    ('FTSE', 'Financial Times Stock Exchange Index', 'GBP', 1.25),
    ('GDAXI', 'German Stock Index', 'EUR', 1.03),
    ('HSI', 'Hang Seng Index', 'HKD', 0.13),
    ('NDX', 'NASDAQ-100 Index', 'USD', 1.00),
    ('RUT', 'Russell 2000 Index', 'USD', 1.00),
    ('SPX', 'S&P 500 Index', 'USD', 1.00);
"""

# Execute the insertion
cursor.execute(insert_data_query)
conn.commit()

print("Ticker data table populated successfully.")


Ticker data table populated successfully.


In [10]:
%%sql 
SELECT * FROM ticker_data

 * sqlite:///data_database.db
Done.


Ticker,Description,Currency,ConversionToUSD
BVSP,Brazilian Stock Index,BRL,0.17
DJI,Dow Jones Industrial Average,USD,1.0
FTSE,Financial Times Stock Exchange Index,GBP,1.25
GDAXI,German Stock Index,EUR,1.03
HSI,Hang Seng Index,HKD,0.13
NDX,NASDAQ-100 Index,USD,1.0
RUT,Russell 2000 Index,USD,1.0
SPX,S&P 500 Index,USD,1.0


In [11]:
%%sql
SELECT tb.Date,
    tb.Ticker,  
    md.Description,
    md.Currency, 
    md.ConversionToUSD,
    tb.AdjustedClose,
    tb.Close,
    ROUND(tb.Close * md.ConversionToUSD,2) AS Close_in_USD,
    tb.Volume,
    ROUND((tb.Close/tb.PrevMonthClose)-1,2) AS PriceChange,
    ROUND((tb.Volume/tb.PrevMonthVolume)-1,2) AS VolumeChange,
    ROUND(LOG(tb.Close/tb.PrevMonthClose),2) AS MonthlyReturn
    
FROM (
SELECT *,
    CAST(LAG (Close, 1, 0) OVER (PARTITION BY Ticker ORDER BY Date ASC) AS float) AS PrevMonthClose,
    CAST(LAG (Volume, 1, 0) OVER (PARTITION BY Ticker ORDER BY Date ASC)AS float) AS PrevMonthVolume    
FROM DataTable
) AS tb 
LEFT JOIN ticker_data AS md ON md.Ticker= tb.Ticker
LIMIT 10

 * sqlite:///data_database.db
Done.


Date,Ticker,Description,Currency,ConversionToUSD,AdjustedClose,Close,Close_in_USD,Volume,PriceChange,VolumeChange,MonthlyReturn
2015-01-01,BVSP,Brazilian Stock Index,BRL,0.17,46908.0,46908.0,7974.36,77719800,None,None,None
2015-02-01,BVSP,Brazilian Stock Index,BRL,0.17,51583.0,51583.0,8769.11,65310100,0.1,-0.16,0.04
2015-03-01,BVSP,Brazilian Stock Index,BRL,0.17,51150.0,51150.0,8695.5,79890000,-0.01,0.22,-0.0
2015-04-01,BVSP,Brazilian Stock Index,BRL,0.17,56229.0,56229.0,9558.93,81114600,0.1,0.02,0.04
2015-05-01,BVSP,Brazilian Stock Index,BRL,0.17,52760.0,52760.0,8969.2,67845700,-0.06,-0.16,-0.03
2015-06-01,BVSP,Brazilian Stock Index,BRL,0.17,53081.0,53081.0,9023.77,61236200,0.01,-0.1,0.0
2015-07-01,BVSP,Brazilian Stock Index,BRL,0.17,50865.0,50865.0,8647.05,65571800,-0.04,0.07,-0.02
2015-08-01,BVSP,Brazilian Stock Index,BRL,0.17,46626.0,46626.0,7926.42,71135200,-0.08,0.08,-0.04
2015-09-01,BVSP,Brazilian Stock Index,BRL,0.17,45059.0,45059.0,7660.03,79146200,-0.03,0.11,-0.01
2015-10-01,BVSP,Brazilian Stock Index,BRL,0.17,45869.0,45869.0,7797.73,78617500,0.02,-0.01,0.01


In [12]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("data_database.db")

# Define your SQL query
query = """
SELECT tb.Date,
    tb.Ticker,  
    md.Description,
    md.Currency, 
    md.ConversionToUSD,
    tb.AdjustedClose,
    tb.Close,
    ROUND(tb.Close * md.ConversionToUSD,2) AS Close_in_USD,
    tb.Volume,
    ROUND((tb.Close/tb.PrevMonthClose)-1,2) AS PriceChange,
    ROUND((tb.Volume/tb.PrevMonthVolume)-1,2) AS VolumeChange,
    ROUND(LOG(tb.Close/tb.PrevMonthClose),2) AS MonthlyReturn
    
FROM (
SELECT *,
    CAST(LAG (Close, 1, 0) OVER (PARTITION BY Ticker ORDER BY Date ASC) AS float) AS PrevMonthClose,
    CAST(LAG (Volume, 1, 0) OVER (PARTITION BY Ticker ORDER BY Date ASC)AS float) AS PrevMonthVolume    
FROM DataTable
) AS tb 
LEFT JOIN ticker_data AS md ON md.Ticker= tb.Ticker
"""

# Execute the SQL query and load the result into a DataFrame
result_df = pd.read_sql_query(query, conn)

# Save the DataFrame to an Excel file
result_df.to_excel("query_result.xlsx", index=False)

print("Query result saved to 'query_result.xlsx'.")


Query result saved to 'query_result.xlsx'.
